In [1]:
###### Set Up #####
# verify our folder with the data and module assets is installed
# if it is installed make sure it is the latest
!test -e ds-assets && cd ds-assets && git pull && cd ..
# if it is not installed clone it
!test ! -e ds-assets && git clone https://github.com/lutzhamel/ds-assets.git
# point to the folder with the assets
home = "ds-assets/assets/"
import sys
sys.path.append(home)      # add home folder to module search path

Already up to date.


# k-NN Classification

k-NN: **k** **N**earest **N**eighbors

Neighbors-based classification is a type of *instance-based learning*: it does not attempt to construct a general internal model, but simply stores instances of the training data.
Classification is computed from a simple majority vote of the *nearest neighbors of each point*: a query point is assigned the data class which has the most representatives within the nearest neighbors of the point.


## Illustration

Consider the following figure,

<!-- ![knn](assets/knn.png) -->
<center>
<img src="https://raw.githubusercontent.com/lutzhamel/ds-assets/main/assets/knn.png" height="256" width="280">
</center>

We want to assign the sample (green bullet) either to the class of blue squares or to the class of red triangles,

* If k = 3 (solid line circle) it is assigned to the class of red triangles because there are 2 triangles and only 1 square inside the inner circle.

* If k = 5 (dashed line circle) it is assigned to the class of blue squares (3 squares vs. 2 triangles inside the dashed circle).


## k-NN Classification

k-NN classification is a supervised learning algorithm, therefore the training examples are vectors in a multidimensional feature space, each with a class label.
The training phase of the algorithm consists only of storing the feature vectors and class labels of the training samples.
In the classification phase, k is a user-defined constant, and an unlabeled vector (a query or test point) is classified by assigning the label which is most frequent among the k training samples nearest to that query point.
A commonly used distance metric for continuous variables is the Euclidean distance. For discrete variables, such as for text classification, another metric can be used, such as the Hamming distance.


## A Code Example

Let's build an k-NN classifier for the iris dataset.  

**NOTE**: we are not searching for the optimal model, we just want to build a classifier.

In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from confint import classification_confint

# get data
df = pd.read_csv(home+"iris.csv")
X  = df.drop(columns=['id','Species'])
y = df[['Species']]

# set up the model with k=3
model = KNeighborsClassifier(n_neighbors=3)

# do train and test
train_X, test_X, train_y, test_y =\
   train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=5)
model.fit(train_X, train_y['Species']) # KNN wants a series as its target

# evaluate the model
predict_y = model.predict(test_X)
acc = accuracy_score(test_y, predict_y)
lb, ub = classification_confint(acc, test_X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f}, {:3.2f})".format(acc, lb, ub))

Accuracy: 0.93 (0.84, 1.00)


The performance is not bad for a randomly chosen value for k.  

# Model Comparison

Here we are a little bit more careful with our model construction and do a cross-validated grid search for the optimal value of k.
Furthermore we want to see how our optimal k-NN classifier performance stacks up to the performance of a decision tree model in a statistical valid manner.


Let’s work our way through this comparison using the `wdbc` dataset:

* Build optimal k-NN and tree models using grid search
* Compute the accuracy for the classifiers
* Print out the confusion matrix for each classifier
* Print out the confidence interval for each classifier
* Decide if the difference between classifiers is statistically significant or not.

## Set Up

Get our training data and format in way that `sklearn` expects.

In [3]:
# basic data routines
import pandas as pd

# models
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# model evaluation routines
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from confint import classification_confint

In [4]:
# get data
df = pd.read_csv(home+"wdbc.csv").drop(columns=['ID'])

# format training data for sklean
X  = df.drop(columns=['Diagnosis'])
actual_y = df[['Diagnosis']]

## k-NN Classifier

First up is the k-NN classifier.  In order to find the optimal model we set up a grid search over the number of neighbors.  In this case we search the values from 1 to 25.

In [5]:
# KNN
model = KNeighborsClassifier()

# grid search
param_grid = {'n_neighbors': list(range(1,26))}
grid = GridSearchCV(model, param_grid, cv=5)
grid.fit(X, actual_y['Diagnosis']) # KNN wants a series as its target

# accuracy of best model with confidence interval
best_model = grid.best_estimator_
predict_y = best_model.predict(X)
acc = accuracy_score(actual_y, predict_y)
lb,ub = classification_confint(acc,X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

# build the confusion matrix for more detailed error analysis
labels = ['M', 'B']
cm = confusion_matrix(actual_y, predict_y, labels=labels)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("Confusion Matrix:\n{}".format(cm_df))

Accuracy: 0.94 (0.92,0.96)
Confusion Matrix:
     M    B
M  186   26
B    8  349


Let's take a look at the performance data.  In terms of accuracy we see that the best k-NN model has an accuracy of 94% with a confidence interval of (92%, 96%).  From a medical application perspective the confusion matrix is worrisome.  We see that of the 212 malignant samples the model misclassifies 26 as benign.  This kind of error is called the 'false negative' error and in this case would mean that 12% of the malignant cases remain undetected. We also see that of the 357 benign samples it misclassifies 8 as malignant.  The is called the 'false positive' error. From a medical point of view this is not as worrisome because additional tests will identify these cases correctly as benign.

## Decision Trees

For decision trees we set up a grid search over the tree depth from 1 to 20 and the criterion which searches over `entropy` and `gini`.

In [6]:
# decision trees
model = DecisionTreeClassifier(random_state=1)

# grid search
param_grid = {'max_depth': list(range(1,21)), 'criterion': ['entropy','gini'] }
grid = GridSearchCV(model, param_grid, cv=5)
grid.fit(X, actual_y)

# accuracy of best model with confidence interval
best_model = grid.best_estimator_
predict_y = best_model.predict(X)
acc = accuracy_score(actual_y, predict_y)
lb,ub = classification_confint(acc,X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

# build the confusion matrix for a more detailed error analysis
labels = ['M', 'B']
cm = confusion_matrix(actual_y, predict_y, labels=labels)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("Confusion Matrix:\n{}".format(cm_df))

Accuracy: 0.98 (0.97,0.99)
Confusion Matrix:
     M    B
M  210    2
B    7  350


The performance of the decision tree model is much better overall and from a medical point specifically.  Less than 1% of the malignant cases is classified as a 'false negative' giving much more confidence in its applicability in a medical setting. The accuracy of the model is 98% with a confidence interval of (97%, 99%).

## Performance Comparison and Model Selection

If we compare models we have to look beyond the raw performance numbers in this case 94% and 98% for k-NN and the decision tree model, respectively. We have to ask if the difference in performance between these two models is statistically significant.  Consider the performance of the k-NN model with an accuracy and confidence interval of,
```
94% (92%, 96%)
```
Also consider the performance of the decision tree model with an accuracy and confidence interval of,
```
98% (97%, 99%)
```
Here we see that
the confidence intervals for the decision tree and the K-NN classifier **do not overlap**.  That means here the decision tree is truly the better model and the performance difference between the two models is **statistically significant**.  

**Observation**: Therefore we will select the **decision tree as a model** for our breast cancer data.
